In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

%load_ext autoreload
%autoreload 2

# making all pandas columns visable with display command
pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings('ignore')

In [2]:
GW7 = pd.read_csv('GW7.csv')
GW7

,HomeTeam,AwayTeam,FTR,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,HPts,APts,PtsDiff5,PtsDiff10,PtsDiff20,GSDiff5,GSDiff10,GSDiff20,GCDiff5,GCDiff10,GCDiff20,SDiff5,SDiff10,SDiff20,STDiff5,STDiff10,STDiff20,FDiff5,FDiff10,FDiff20,CDiff5,CDiff10,CDiff20,YDiff5,YDiff10,YDiff20,RDiff5,RDiff10,RDiff20,H1,A1,HWinStreak3,AWinStreak3,HWinStreak5,AWinStreak5,HLossStreak3,ALossStreak3,HLossStreak5,ALossStreak5,GDDiff5,GDDiff10,GDDiff20
0,Aston Villa,Brighton,H,6.0,1.0,19.0,11.0,9.0,3.0,19.0,21.0,1.0,3.0,4.0,4.0,0.0,0.0,3.0,0.0,0.0,0.000000,0.200000,-0.6,-0.700000,-0.600000,-0.4,-0.2,-0.60,-2.0,-4.1,-6.05,-1.6,-2.000000,-2.800000,-1.2,-1.000000,-1.550000,0.2,0.900000,-0.650000,-0.8,0.1,0.05,0.0,0.0,0.00,3,3,0,1,0,0,0,0,0,0,-0.2,-0.500000,0.000000
1,Bournemouth,Arsenal,A,0.0,4.0,8.0,15.0,1.0,8.0,11.0,10.0,6.0,6.0,0.0,1.0,0.0,0.0,0.0,3.0,-1.8,-1.700000,-1.000000,-1.0,-1.200000,-1.350000,1.0,0.8,0.40,-3.0,-3.3,-3.80,-1.6,-1.900000,-1.650000,6.0,2.100000,1.800000,-7.0,-4.100000,-3.150000,0.6,0.5,0.80,-0.2,-0.1,-0.05,0,1,0,0,0,0,0,0,0,0,-2.0,-2.000000,-1.750000
2,Everton,Luton,A,1.0,2.0,23.0,9.0,5.0,2.0,8.0,10.0,3.0,6.0,0.0,1.0,0.0,0.0,0.0,3.0,0.6,0.900000,0.750000,0.4,0.600000,0.500000,-0.4,-0.7,-0.45,0.6,0.8,0.80,2.2,2.700000,2.700000,0.2,0.100000,0.000000,-1.6,-0.500000,-1.700000,0.8,0.4,0.30,0.0,0.0,0.10,3,1,0,0,0,0,0,0,0,0,0.8,1.300000,0.950000
3,Man United,Crystal Palace,A,0.0,1.0,19.0,8.0,4.0,2.0,10.0,8.0,10.0,4.0,3.0,2.0,0.0,0.0,0.0,3.0,0.2,0.800000,0.450000,0.2,0.500000,-0.050000,0.6,0.1,0.00,3.4,4.3,3.90,0.2,1.100000,1.350000,-2.4,-0.700000,-1.100000,3.4,2.100000,1.500000,1.0,0.7,0.40,0.0,0.0,-0.05,3,1,0,0,0,0,0,0,0,0,-0.4,0.400000,-0.050000
4,Newcastle,Burnley,H,2.0,0.0,20.0,8.0,8.0,2.0,7.0,15.0,5.0,3.0,1.0,3.0,0.0,0.0,3.0,0.0,1.0,1.000000,1.150000,1.4,1.400000,1.600000,-1.4,-1.0,-0.70,3.4,4.9,5.60,2.8,2.600000,2.900000,1.2,1.100000,0.550000,-1.6,-0.100000,1.050000,0.6,0.6,-0.15,-0.4,-0.3,-0.20,3,0,0,0,0,0,0,0,0,0,2.8,2.400000,2.300000
5,West Ham,Sheffield United,H,2.0,0.0,20.0,16.0,9.0,2.0,6.0,9.0,7.0,4.0,1.0,1.0,0.0,0.0,3.0,0.0,1.6,0.900000,0.700000,1.0,0.900000,1.000000,-1.4,-0.5,-0.20,1.6,3.0,3.20,0.0,1.500000,1.800000,-0.6,-2.300000,-2.250000,1.0,0.900000,1.300000,-0.8,-1.0,-0.75,0.0,0.0,-0.05,0,0,0,0,0,0,0,0,0,0,2.4,1.400000,1.200000
6,Wolves,Man City,H,2.0,1.0,3.0,23.0,1.0,8.0,18.0,12.0,0.0,6.0,5.0,4.0,0.0,0.0,3.0,0.0,-2.2,-1.700000,-1.700000,-1.4,-1.300000,-1.550000,1.6,1.5,1.25,-6.8,-5.2,-4.70,-4.2,-2.900000,-2.600000,5.0,5.000000,4.200000,-4.8,-2.100000,-1.900000,1.0,1.7,1.35,0.2,0.1,0.10,1,3,0,1,0,1,0,0,0,0,-3.0,-2.800000,-2.800000
7,Tottenham,Liverpool,H,2.0,1.0,24.0,12.0,8.0,4.0,11.0,17.0,12.0,5.0,5.0,4.0,0.0,2.0,3.0,0.0,-0.4,-0.400000,-0.600000,-0.2,-0.200000,-0.450000,0.2,0.3,0.55,1.0,-1.5,-0.85,1.2,0.800000,0.400000,2.4,-0.400000,-0.200000,-1.4,-1.200000,-1.100000,2.2,0.9,0.65,-0.4,-0.2,-0.05,1,3,0,1,0,1,0,0,0,0,-0.4,-0.500000,-1.000000
8,Nott'm Forest,Brentford,D,1.0,1.0,6.0,18.0,1.0,5.0,6.0,10.0,1.0,11.0,1.0,2.0,1.0,0.0,1.0,1.0,0.4,0.000000,-0.350000,-0.2,0.000000,0.000000,0.0,0.4,0.60,-3.4,-4.0,-2.10,-0.4,-1.300000,-0.900000,2.0,2.900000,2.550000,-0.4,-0.300000,-0.400000,1.8,0.9,1.10,0.2,0.1,0.00,0,0,0,0,0,0,0,0,0,0,-0.2,-0.400000,-0.600000
9,Fulham,Chelsea,A,0.0,2.0,10.0,11.0,3.0,4.0,15.0,12.0,8.0,1.0,1.0,4.0,0.0,0.0,0.0,3.0,0.2,0.800000,0.200000,0.0,0.600000,0.250000,1.0,0.3,0.25,-6.8,-5.1,-4.25,-1.6,-1.400000,-1.200000,-2.4,0.000000,0.850000,-2.8,-1.100000,-0.950000,-0.4,-0.4,-0.55,0.4,0.2,0.10,1,0,0,0,0,0,0,0,0,0,-1.0,0.300000,0.000000


In [3]:
GW7 = GW7[['PtsDiff5','PtsDiff10','PtsDiff20',
         'GSDiff5','GSDiff10','GSDiff20',
         'GCDiff5','GCDiff10','GCDiff20',
         'SDiff5','SDiff10','SDiff20',
         'STDiff5','STDiff10','STDiff20',
         'FDiff5','FDiff10','FDiff20',
         'CDiff5','CDiff10','CDiff20',
         'YDiff5','YDiff10','YDiff20',
         'RDiff5','RDiff10','RDiff20',
         'H1','A1',
         'HWinStreak3','AWinStreak3','HWinStreak5','AWinStreak5',
         'HLossStreak3','ALossStreak3','HLossStreak3','ALossStreak5',
         'GDDiff5','GDDiff10','GDDiff20']]

In [4]:
GW7

,PtsDiff5,PtsDiff10,PtsDiff20,GSDiff5,GSDiff10,GSDiff20,GCDiff5,GCDiff10,GCDiff20,SDiff5,SDiff10,SDiff20,STDiff5,STDiff10,STDiff20,FDiff5,FDiff10,FDiff20,CDiff5,CDiff10,CDiff20,YDiff5,YDiff10,YDiff20,RDiff5,RDiff10,RDiff20,H1,A1,HWinStreak3,AWinStreak3,HWinStreak5,AWinStreak5,HLossStreak3,ALossStreak3,HLossStreak3,ALossStreak5,GDDiff5,GDDiff10,GDDiff20
0,0.0,0.000000,0.200000,-0.6,-0.700000,-0.600000,-0.4,-0.2,-0.60,-2.0,-4.1,-6.05,-1.6,-2.000000,-2.800000,-1.2,-1.000000,-1.550000,0.2,0.900000,-0.650000,-0.8,0.1,0.05,0.0,0.0,0.00,3,3,0,1,0,0,0,0,0,0,-0.2,-0.500000,0.000000
1,-1.8,-1.700000,-1.000000,-1.0,-1.200000,-1.350000,1.0,0.8,0.40,-3.0,-3.3,-3.80,-1.6,-1.900000,-1.650000,6.0,2.100000,1.800000,-7.0,-4.100000,-3.150000,0.6,0.5,0.80,-0.2,-0.1,-0.05,0,1,0,0,0,0,0,0,0,0,-2.0,-2.000000,-1.750000
2,0.6,0.900000,0.750000,0.4,0.600000,0.500000,-0.4,-0.7,-0.45,0.6,0.8,0.80,2.2,2.700000,2.700000,0.2,0.100000,0.000000,-1.6,-0.500000,-1.700000,0.8,0.4,0.30,0.0,0.0,0.10,3,1,0,0,0,0,0,0,0,0,0.8,1.300000,0.950000
3,0.2,0.800000,0.450000,0.2,0.500000,-0.050000,0.6,0.1,0.00,3.4,4.3,3.90,0.2,1.100000,1.350000,-2.4,-0.700000,-1.100000,3.4,2.100000,1.500000,1.0,0.7,0.40,0.0,0.0,-0.05,3,1,0,0,0,0,0,0,0,0,-0.4,0.400000,-0.050000
4,1.0,1.000000,1.150000,1.4,1.400000,1.600000,-1.4,-1.0,-0.70,3.4,4.9,5.60,2.8,2.600000,2.900000,1.2,1.100000,0.550000,-1.6,-0.100000,1.050000,0.6,0.6,-0.15,-0.4,-0.3,-0.20,3,0,0,0,0,0,0,0,0,0,2.8,2.400000,2.300000
5,1.6,0.900000,0.700000,1.0,0.900000,1.000000,-1.4,-0.5,-0.20,1.6,3.0,3.20,0.0,1.500000,1.800000,-0.6,-2.300000,-2.250000,1.0,0.900000,1.300000,-0.8,-1.0,-0.75,0.0,0.0,-0.05,0,0,0,0,0,0,0,0,0,0,2.4,1.400000,1.200000
6,-2.2,-1.700000,-1.700000,-1.4,-1.300000,-1.550000,1.6,1.5,1.25,-6.8,-5.2,-4.70,-4.2,-2.900000,-2.600000,5.0,5.000000,4.200000,-4.8,-2.100000,-1.900000,1.0,1.7,1.35,0.2,0.1,0.10,1,3,0,1,0,1,0,0,0,0,-3.0,-2.800000,-2.800000
7,-0.4,-0.400000,-0.600000,-0.2,-0.200000,-0.450000,0.2,0.3,0.55,1.0,-1.5,-0.85,1.2,0.800000,0.400000,2.4,-0.400000,-0.200000,-1.4,-1.200000,-1.100000,2.2,0.9,0.65,-0.4,-0.2,-0.05,1,3,0,1,0,1,0,0,0,0,-0.4,-0.500000,-1.000000
8,0.4,0.000000,-0.350000,-0.2,0.000000,0.000000,0.0,0.4,0.60,-3.4,-4.0,-2.10,-0.4,-1.300000,-0.900000,2.0,2.900000,2.550000,-0.4,-0.300000,-0.400000,1.8,0.9,1.10,0.2,0.1,0.00,0,0,0,0,0,0,0,0,0,0,-0.2,-0.400000,-0.600000
9,0.2,0.800000,0.200000,0.0,0.600000,0.250000,1.0,0.3,0.25,-6.8,-5.1,-4.25,-1.6,-1.400000,-1.200000,-2.4,0.000000,0.850000,-2.8,-1.100000,-0.950000,-0.4,-0.4,-0.55,0.4,0.2,0.10,1,0,0,0,0,0,0,0,0,0,-1.0,0.300000,0.000000


In [5]:
diff_model = load_model('first_diff_model.h5')
diff_model_2 = load_model('second_diff_model.h5')
diff_model_3 = load_model('third_diff_model.h5')

In [6]:
GW7_predictions = np.argmax(diff_model.predict(GW7), axis=1)
GW7_predictions

1/1 [==============================] - 0s 175ms/step


array([2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2], dtype=int64)

In [7]:
diff_model.predict(GW7)

1/1 [==============================] - 0s 23ms/step


array([[3.53014767e-01, 2.84575075e-01, 3.61971259e-01, 3.21280572e-06,
        3.69769145e-06, 5.09335268e-06, 4.22416088e-06, 5.26469421e-06,
        8.71363682e-06, 5.41297959e-06, 9.24150845e-06, 6.44407419e-06,
        6.06282856e-06, 6.31539160e-06, 6.25246184e-06, 4.05515539e-06,
        1.42587460e-05, 4.68473581e-06, 4.85547798e-06, 7.55792189e-06,
        4.11571273e-06, 7.96104359e-06, 1.45640524e-05, 5.47563195e-06,
        5.66210110e-06, 9.49538389e-06, 5.20591448e-06, 4.92414847e-06,
        4.76909509e-06, 3.22960636e-06, 2.79990854e-06, 3.15125953e-06,
        1.06104071e-05, 2.75423463e-06, 1.03809207e-05, 6.91865353e-06,
        5.65981736e-06, 9.58161036e-06, 4.15275781e-06, 7.24955953e-06,
        4.60238198e-06, 6.61576041e-06, 4.45718024e-06, 3.04710466e-06,
        8.39277072e-06, 8.47078627e-06, 3.11126882e-06, 3.96826590e-06,
        7.15080523e-06, 1.48256331e-05, 8.45760405e-06, 5.14368912e-06,
        8.97789141e-06, 6.79262985e-06, 2.82498900e-06, 5.823885

In [8]:
GW7_predictions_2 = np.argmax(diff_model_2.predict(GW7), axis=1)
GW7_predictions_2

1/1 [==============================] - 0s 80ms/step


array([0, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2], dtype=int64)

In [9]:
diff_model_2.predict(GW7)

1/1 [==============================] - 0s 22ms/step


array([[0.41538683, 0.30738974, 0.2772235 ],
       [0.6592414 , 0.20686878, 0.13388985],
       [0.2164555 , 0.21026595, 0.57327855],
       [0.14318644, 0.26752365, 0.58928984],
       [0.07830618, 0.12055844, 0.8011354 ],
       [0.14031021, 0.22866824, 0.6310216 ],
       [0.70654076, 0.20822953, 0.08522972],
       [0.44068563, 0.26174012, 0.29757425],
       [0.36130816, 0.2566781 , 0.38201368],
       [0.34126443, 0.2632936 , 0.39544198],
       [0.22402266, 0.26423758, 0.5117397 ]], dtype=float32)

In [10]:
GW7_predictions_3 = np.argmax(diff_model_3.predict(GW7), axis=1)
GW7_predictions_3

1/1 [==============================] - 0s 70ms/step


array([0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2], dtype=int64)

In [11]:
diff_model_3.predict(GW7)

1/1 [==============================] - 0s 35ms/step


array([[0.3947885 , 0.26944175, 0.33576962],
       [0.55166286, 0.24641235, 0.2019248 ],
       [0.2707774 , 0.2671312 , 0.4620913 ],
       [0.23862682, 0.2680483 , 0.49332482],
       [0.11971943, 0.18480782, 0.69547266],
       [0.17701203, 0.24831064, 0.5746773 ],
       [0.56847686, 0.22523311, 0.20629019],
       [0.37843838, 0.24471112, 0.3768505 ],
       [0.39379364, 0.2741715 , 0.33203492],
       [0.4097234 , 0.27412504, 0.31615153],
       [0.27366063, 0.27197933, 0.45436   ]], dtype=float32)